# MEIRL

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
area = "london"
filename = "data/" + area + "/adjacency_matrix.csv"
adjacency_matrix = pd.read_csv(filename, index_col=0)

In [3]:
expert_feature_count = adjacency_matrix.copy()
expert_feature_count["value"] = 0
expert_feature_count = expert_feature_count["value"]
expert_feature_count = expert_feature_count.to_frame()
learner_feature_count = expert_feature_count.copy()
test_feature_count = expert_feature_count.copy()

In [4]:
# cross_validation
all_data = 400
training_number = int(all_data * 0.75)
test_number = int(all_data * 0.25)

# Import expert trajectory

### prepare dataset for training

In [5]:
for i in range (1,training_number):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(expert_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        expert_feature_count.loc[int(j), ["value"]] = new_count
expert_feature_count = (expert_feature_count - expert_feature_count.min()) / expert_feature_count.max() - expert_feature_count.min()    

### prepare for testing

In [6]:
for i in range (training_number + 1 , all_data):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(test_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        test_feature_count.loc[int(j), ["value"]] = new_count
test_feature_count = (test_feature_count - test_feature_count.min()) / test_feature_count.max() - test_feature_count.min()    

# Let's train

In [36]:
action_list = np.array(adjacency_matrix[adjacency_matrix[current_position] == 1].index)
value_list = reward_parameter[reward_parameter.index.isin(action_list)]

In [47]:
total_reward_lists = []
for i in range(len(action_list)):

    immediate_reward = value_list.values[i][0]
    one_step_position = value_list.index[i]
    one_step_action_list = np.array(adjacency_matrix[adjacency_matrix[str(one_step_position)] == 1].index)
    one_step_value_list = reward_parameter[reward_parameter.index.isin(one_step_action_list)]
    for j in range(len(one_step_value_list)):
        total_reward_list = []
        one_step_reward = one_step_value_list.values[j][0]
        total_reward = immediate_reward + one_step_reward
        total_reward_list = [total_reward, i, j]
        total_reward_lists.append(total_reward_list)
total_reward_lists = np.array(total_reward_lists)
highest_reward = np.argmax(total_reward_lists[:,0])
updated_position = action_list[int(total_reward_lists[highest_reward][1])]           



In [13]:
#Initialization of reward parameter
reward_parameter = expert_feature_count.copy()
reward_parameter.columns = ["value"]
reward_parameter["value"] = np.random.random_sample(reward_parameter.shape[0])

In [33]:
def update_position(current_position, eps, reward):
    
    action_list = np.array(adjacency_matrix[adjacency_matrix[current_position] == 1].index)
    value_list = reward[reward.index.isin(action_list)]

    if np.random.random() > eps:
        
        total_reward_lists = []
        for i in range(len(action_list)):
            immediate_reward = value_list.values[i][0]
            one_step_position = value_list.index[i]
            one_step_action_list = np.array(adjacency_matrix[adjacency_matrix[str(one_step_position)] == 1].index)
            one_step_value_list = reward[reward.index.isin(one_step_action_list)]
            for j in range(len(one_step_value_list)):
                total_reward_list = []
                one_step_reward = one_step_value_list.values[j][0]
                total_reward = immediate_reward + one_step_reward
                total_reward_list = [total_reward, i, j]
                total_reward_lists.append(total_reward_list)
        total_reward_lists = np.array(total_reward_lists)
        highest_reward = np.argmax(total_reward_lists[:,0])
        
        
        hoge = int(total_reward_lists[highest_reward][1])
        
        action = action_list(hoge)

        updated_position = action_list[ac]

    else:
        updated_position = random.choice(value_list["value"].index.tolist())

    return updated_position

In [34]:
# how many iterations?
Y = []
iteration = 50
for i in range (iteration):
    epsilon = 1 / (i+1)  
    learner_feature_count["value"] = 0
    for j in range (1,training_number):
        start_position = random.choice(adjacency_matrix.columns)
        current_position = start_position
        # agent will walk for 50 steps
        for k in range(50):
            current_position = str(update_position(current_position, epsilon, reward_parameter))
            learner_feature_count.loc[int(current_position), ["value"]] += 1
    
    learner_feature_count = (learner_feature_count - learner_feature_count.min()) / (learner_feature_count.max() - learner_feature_count.min() )   
    gradient = (expert_feature_count - learner_feature_count) 
    reward_parameter = (gradient*0.2) + reward_parameter
    reward_parameter = (reward_parameter - reward_parameter.min()) / (reward_parameter.max() - reward_parameter.min() )   
    
    difference = gradient.abs().sum().values[0]
    Y.append(difference)
    print(i, difference)
    
    correlation_dataframe = pd.concat([learner_feature_count, test_feature_count], axis=1)
    correlation = correlation_dataframe.corr().values[0][1]
    print(correlation)

0 211.701754386
0.0552857603897


TypeError: 'numpy.ndarray' object is not callable

# Local Depth

In [8]:
#local depth
filename = "data/" + area + "/" + "closeness_centrality.csv"

closeness_centrality = pd.read_csv(filename, index_col=0)
adjacent_node = adjacency_matrix.sum().to_frame()
adjacent_node.columns = ["closeness_centrality"]
adjacent_node.index = adjacent_node.index.map(int)
mean_depth = (closeness_centrality - adjacent_node) / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   

In [9]:
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_local_depth = - correlation_dataframe.corr().values[0][1]

In [10]:
correlation_local_depth

0.45406804231592879

# Connectivity 

In [11]:
mean_depth = adjacent_node / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_connectivity = correlation_dataframe.corr().values[0][1]
correlation_connectivity

0.32460389310488996

# Global Depth

In [12]:
mean_depth = closeness_centrality / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_global_depth = - correlation_dataframe.corr().values[0][1]
correlation_global_depth

0.45406704994670616